In [1]:
import OpenEphys
import Kwik
from matplotlib import pyplot as plt
import numpy as np
import os
import re


In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [3]:
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-02/2019-02-02_16-00-36/'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-02/2019-02-02_14-28-03/'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-02/2019-02-02_15-43-35/'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-02/2019-02-02_16-00-36/'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-02/2019-02-02_16-06-41/'  <<<< KWIK
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-02/2019-02-02_16-22-41/'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2018-11-27_17-21-05/experiment1_103.raw.kwd'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2018-11-27_17-38-51/experiment1_103.raw.kwd'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-06_09-22-17/2019-02-06_13-19-58/'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-06_09-22-17/2019-02-06_13-06-08/'
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-08/2019-02-08_17-26-53/'
source_path = '../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/'

In [4]:
f_names = []
for root, dirnames, filenames in os.walk(source_path):
    for filename in filenames:
#         f_regex = re.compile(r".*CH(\d|\d\d)\.continuous$")
#         f_regex = re.compile(r".*CH(\d)\.continuous$")
        f_regex = re.compile(r".*CH(1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16)\.continuous$")
#         f_regex = re.compile(r".*\.continuous$")
#         f_regex = re.compile(r".*_3\.continuous$")
        if f_regex.match(filename):
            f_names = f_names + [os.path.join(root, filename)]

In [5]:
for i,fn in enumerate(f_names):
    print('{}: {}'.format(i,fn))
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-02/2019-02-02_15-43-35/' _3 has good CS
# source_path = '/mnt/data/temp/kaveh/openEphys_data/2019-02-02/2019-02-02_16-00-36/' mehhh

0: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH7.continuous
1: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH9.continuous
2: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH5.continuous
3: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH8.continuous
4: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH11.continuous
5: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH12.continuous
6: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH14.continuous
7: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH10.continuous
8: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH16.continuous
9: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH4.continuous
10: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH2.continuous
11: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH15.continuous
12: ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH3.continuous
13:

In [1]:
import scipy.signal
import scipy.fftpack

def common_avg_ref(data, refrence, affected):
    signal_average = np.mean(data[refrence, :], axis = 0)
    data[affected,:] = data[affected,:] - signal_average

def notch_all_harmonics(signal, base_freq, sampling_rate):
    """
    Stoppass filter at the base_freq and all of its harmonics
    """
    if base_freq > sampling_rate/2.0:
        print('Invalid frequency to notch')
        return
    curr_freq = base_freq
    while curr_freq < sampling_rate/2.0:
        b, a = scipy.signal.iirnotch(curr_freq/(sampling_rate/2.0), 30)
        signal = scipy.signal.lfilter(b, a, signal)
        curr_freq = curr_freq + base_freq
        
    return signal
        

def butter_lowpass_filter(data, cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scipy.signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = scipy.signal.lfilter(b, a, data)
    return y


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = scipy.signal.butter(order, [low, high], btype='band')
    y = scipy.signal.lfilter(b, a, data)
    return y


In [8]:
Fs

30000.0

In [7]:
#TODO: if open ephys
import scipy.io

all_channels = []
all_channels_filtered = []
all_channels_lfp = []
all_fs = []
for i, fn in enumerate(f_names):
    print('{}: reading {}'.format(i,fn))
    file_content = OpenEphys.load(fn)
    Fs = float(file_content['header']['sampleRate'])
    signal_filtered = butter_bandpass_filter(file_content['data'], 300, 3000, Fs, order=2 )
    signal_lfp = butter_bandpass_filter(file_content['data'], 0.01, 300, Fs, order=2 )
#     all_channels = all_channels + [file_content['data']]
    all_channels_filtered = all_channels_filtered + [signal_filtered]
    all_channels_lfp = all_channels_lfp + [signal_lfp]
#     scipy.io.savemat(source_path + 'channel_{}.mat'.format(i), {'channel_{}'.format(i): file_content['data']})
    print('Length of signal is {}'.format(file_content['data'].shape))
    all_fs = all_fs + [float(file_content['header']['sampleRate'])]
all_channels = np.array(all_channels)
all_channels_filtered = np.array(all_channels_filtered)
all_channels_lfp = np.array(all_channels_lfp)
all_fs = np.array(all_fs)



0: reading ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH7.continuous
Loading continuous data...
Length of signal is (36038656,)
1: reading ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH9.continuous
Loading continuous data...
Length of signal is (36038656,)
2: reading ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH5.continuous
Loading continuous data...
Length of signal is (36038656,)
3: reading ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH8.continuous
Loading continuous data...
Length of signal is (36038656,)
4: reading ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH11.continuous
Loading continuous data...
Length of signal is (36038656,)
5: reading ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH12.continuous
Loading continuous data...
Length of signal is (36038656,)
6: reading ../data/OpenEphys_data/2019-04-26/2019-04-26_12-03-55/101_CH14.continuous
Loading continuous data...
Length of signal is (3603865

In [ ]:
#TODO: if KWIK
# all_channels = np.transpose(Kwik.load(source_path)['data'][:,0:4])

In [ ]:
print(all_channels_filtered.shape)
print(all_channels_lfp.shape)
print(all_channels.shape)

In [ ]:
np.arange(0,num_chans)

In [ ]:
num_chans = 6
# common_avg_ref(all_channels, np.arange(0,4), np.arange(0,4))
common_avg_ref(all_channels_lfp, np.arange(0,num_chans), np.arange(0,num_chans))
common_avg_ref(all_channels_filtered, np.arange(0,num_chans), np.arange(0,num_chans))

In [ ]:
chn = 2
plt.figure(figsize=(10,5))
# plt.plot(np.arange(0, all_channels_lfp.shape[1]) * (1/all_fs[chn]), all_channels_lfp[chn,:].T, alpha=0.3, color='r', zorder=10)
# plt.figure(figsize=(15,3))
plt.plot(np.arange(0, all_channels_filtered.shape[1]) * (1/all_fs[chn]), all_channels_filtered[chn,:].T, alpha=0.8, color='k')
plt.xlabel('Time (s)')
plt.show()
# plt.plot(signal_average[0:1000], 'k')

In [ ]:
all_channels_filtered = []

In [ ]:
import gc
gc.collect()

In [ ]:
chn = -1
voltage = all_channels_filtered[chn,:].T

In [ ]:
dt = (1/all_fs[chn])

In [ ]:
from kaveh.sorting.spikesorter import SimpleSpikeSorter


In [ ]:
sss = SimpleSpikeSorter(voltage, dt)

In [ ]:
sss._pre_process()

In [ ]:
sss._detect_spikes_minibatch()

In [ ]:
sss._align_spikes()


In [ ]:
sss.cs_num_gmm_components = 5

In [ ]:
# sss.spike_indices = spike_indices
sss._cluster_spike_by_feature()
sss._cs_post_process()
print(sss.cs_indices.shape)

In [ ]:
from sklearn.mixture import GaussianMixture

pre_time = 0.0005
post_time = 0.005

pre_index = int(np.round(pre_time/sss.dt))
post_index = int(np.round(post_time/sss.dt))
aligned_cs = np.array([sss.voltage[i - pre_index : i + post_index] for i in sss.cs_indices])


import random

ss_indices = np.setdiff1d(sss.spike_indices, sss.cs_indices)
aligned_ss = np.array([sss.voltage[i - pre_index : i + post_index] for i in ss_indices[1:-2]])


mean_ss = np.mean(aligned_ss[random.sample(range(0, aligned_ss.shape[0]), sss.cs_indices.size), ], axis=0)

In [ ]:
num_clusters = 8
gmm = GaussianMixture(num_clusters, covariance_type = 'full').fit(aligned_cs)

cluster_labels = gmm.predict(aligned_cs)

clusters = []
for cn in np.arange(num_clusters):
    clusters.append(aligned_cs[np.where(cluster_labels == cn)])
    

#         mean_ss = np.mean(aligned_ss, axis=0)

In [ ]:
colors = plt.cm.nipy_spectral(np.linspace(0,1,num_clusters))
legend_labels = []
for cn in np.arange(num_clusters):
    legend_labels.append('c{}({}) '.format(cn, clusters[cn].shape[0]))

import gc
gc.collect()
plt.figure(figsize=(8,5))
# ax2 = plt.subplot(122)
for cn in np.arange(num_clusters):
    plt.plot(np.mean(clusters[cn], axis=0), color = colors[cn], label = legend_labels[cn])
plt.plot(mean_ss, '--', label = 'Mean SS({})'.format(aligned_ss.shape[0]))
# plt.show()    
plt.legend() 
# plt.title('{}; length: {}s'.format(os.path.basename(file_names[f_index]),sss.voltage.shape[0]*sss.dt))

In [ ]:
plt.title('Signal length = {}s'.format(voltage.size*dt))

In [ ]:
# plt.savefig('../data/plots/ehsan_neuron_2019-04-12_13-12-35-ch1.png')

In [ ]:
plt.figure()
plt.plot( all_channels_filtered[chn,:].T, alpha=0.8, color='k')
plt.eventplot(sss.cs_indices[np.where(cluster_labels == 0)],linelengths=300 , colors='red', alpha = 0.3 , zorder = 10)
plt.xlabel('Time (s)')
plt.show()


In [ ]:
# plt.figure(figsize=(5, 25))
# for row in all_channels_filtered:
#      plt.plot(row + 800)
# #     print(np.max(all_channels_filtered[i,:]))


# Three subplots sharing both x/y axes
f, axes = plt.subplots(16, sharex=True, sharey=True)
for i, row in enumerate(all_channels_filtered):

    axes[i].plot(np.arange(0, row.size) * (1/all_fs[i]), row)
    
    # Fine-tune figure; make subplots close to each other and hide x ticks for
    # all but bottom plot.
    f.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)

In [ ]:
np.min(all_channels_lfp[chn,:])

In [ ]:
chn = 2
plt.figure(figsize=(15,3))
plt.plot(np.arange(0, all_channels.shape[1]) * (1/all_fs[chn]), all_channels[chn,:].T, alpha=0.5, color='b')
plt.plot(np.arange(0, all_channels_filtered.shape[1]) * (1/all_fs[chn]), all_channels_filtered[chn,:].T, alpha=0.5, color='r')
plt.xlabel('Time (s)')
plt.show()